In [1]:
import json
from bs4 import Tag
from tqdm import tqdm
import tbmgar_utils
import os
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:

class CustomJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Tag):
            return obj.get_text()
        return str(obj)

def save_json(path, file_name, data):
    try: 
        with open(path + file_name, "w", encoding='utf-8') as outfile:
            json.dump(data, outfile, indent=4, ensure_ascii=False, cls=CustomJSONEncoder)
        print(f"Successfully saved: {file_name}")
    except Exception as e:
        print(f"Error saving {file_name}: {str(e)}")

def scrape_article(url, page_key_code):
    try:
        article_content = tbmgar_utils.scrape_tbmgar_article_content(url, tags=page_key_code)
        return article_content
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
        return None

def get_content(All_links_data, Total_lenght, page_key_code, page_key_list):
    all_article = {}
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_url = {}
        start = 1
        for i in range(start, Total_lenght+1):
            page_key = page_key_code + str(i)
            all_link_page = All_links_data[page_key]["Links"]
            for url in all_link_page:
                future = executor.submit(scrape_article, url, page_key_code)
                future_to_url[future] = (page_key, url)
        
        for future in tqdm(as_completed(future_to_url), total=len(future_to_url)):
            page_key, url = future_to_url[future]
            try:
                article_content = future.result()
                if article_content:
                    article_key = f"{page_key}_tbmgar_Article_{len([k for k in all_article if k.startswith(page_key)]) + 1}"
                    all_article[article_key] = article_content
            except Exception as e:
                print(f"Error processing {url}: {str(e)}")
    
    Failure_count = sum(1 for article in all_article.values() if article["Response"] != 200)
    print(f"Total Failure in the {page_key_list[1]} article: {Failure_count}")
    
    save_file_name = f"tbmgar_ALL_content_{page_key_list[1]}.json"
    print(save_file_name)
    path = "./data/parallel_content/"
    save_json(path, save_file_name, all_article)

def process_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            All_links_data = json.load(file)
            Total_lenght = len(All_links_data)
            print(f"Total page in {os.path.basename(file_path)}: {Total_lenght}")
        
            print(f"page key name: {list(All_links_data.keys())[-1]}")
            page_key_list = list(All_links_data.keys())[-1].split(" ")
            
            page_key_code = "Page "+page_key_list[1]+" "
            print(f"Page key code: {page_key_code}")
            get_content(All_links_data, Total_lenght, page_key_code, page_key_list)
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {os.path.basename(file_path)}: {str(e)}")
    except Exception as e:
        print(f"Error processing file {os.path.basename(file_path)}: {str(e)}")

def get_json_files(directory):
    json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        list(tqdm(executor.map(process_json_file, json_files), total=len(json_files)))
    
    print(f"Processed {len(json_files)} files")


In [3]:
%%time
# Example usage
directory_path = './data/links/'
get_json_files(directory_path)

Total page in tbmgar_ALL_link_བོད་ནང་གསར་འགྱུར།.json: 49
page key name: Page བོད་ནང་གསར་འགྱུར། 49
Page key code: Page བོད་ནང་གསར་འགྱུར། 
Total page in tbmgar_ALL_link_ཁཱ་ཤེས་རབ་རྒྱ་མཚོ།.json: 12
page key name: Page ཁཱ་ཤེས་རབ་རྒྱ་མཚོ། 12
Page key code: Page ཁཱ་ཤེས་རབ་རྒྱ་མཚོ། 
Total page in tbmgar_ALL_link_རི་མོ།.json: 1
page key name: Page རི་མོ། 1
Page key code: Page རི་མོ། 
Total page in tbmgar_ALL_link_ཁོར་ཡུག་སྲུང་སྐྱོབ།.json: 3
page key name: Page ཁོར་ཡུག་སྲུང་སྐྱོབ། 3
Page key code: Page ཁོར་ཡུག་སྲུང་སྐྱོབ། 
Total page in tbmgar_ALL_link_བདམས་བཀོད་རྩོམ་ཡིག.json: 13
page key name: Page བདམས་བཀོད་རྩོམ་ཡིག 13
Page key code: Page བདམས་བཀོད་རྩོམ་ཡིག 
Total page in tbmgar_ALL_link_གཏམ་དཔེ།.json: 2
page key name: Page གཏམ་དཔེ། 2
Page key code: Page གཏམ་དཔེ། 
Total page in tbmgar_ALL_link_གནའ་གཏམ་དང་སྒྲུང་།.json: 3
page key name: Page གནའ་གཏམ་དང་སྒྲུང་། 3
Page key code: Page གནའ་གཏམ་དང་སྒྲུང་། 
Total page in tbmgar_ALL_link_མགར་གྱི་མི་སྣ།.json: 1
page key name: Page མགར་གྱི་མི་སྣ། 1
Page

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]


  0%|          | 0/22 [00:00<?, ?it/s]



  0%|          | 0/17 [00:00<?, ?it/s]




  0%|          | 0/25 [00:00<?, ?it/s]





  0%|          | 0/58 [00:00<?, ?it/s]






  0%|          | 0/56 [00:00<?, ?it/s]








  0%|          | 0/127 [00:00<?, ?it/s]









  0%|          | 0/489 [00:00<?, ?it/s]







  0%|          | 0/113 [00:00<?, ?it/s]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


 11%|█         | 1/9 [00:07<01:02,  7.80s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

 10%|█         | 1/10 [00:08<01:14,  8.31s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.




  6%|▌         | 1/17 [00:08<02:15,  8.50s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.










  0%|          | 1/489 [00:10<1:21:41, 10.04s/it]Some characters could not

Total Failure in the རི་མོ། article: 0
tbmgar_ALL_content_རི་མོ།.json
Successfully saved: tbmgar_ALL_content_རི་མོ།.json
Total page in tbmgar_ALL_link_གླུ་དབྱངས།.json: 1
page key name: Page གླུ་དབྱངས། 1
Page key code: Page གླུ་དབྱངས། 


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
  8%|▊         | 9/113 [03:09<32:30, 18.75s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.




Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
 71%|███████   | 12/17 [03:09<01:30, 18.07s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



 45%|████▌     | 10/22 [03:09<03:36, 18.02s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.







Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
 18%|█▊        | 10/56 [03:10<13:35, 17.73s/it]Some characters could not be decoded, and 

Total Failure in the མགར་གྱི་མི་སྣ། article: 0
tbmgar_ALL_content_མགར་གྱི་མི་སྣ།.json


...

Total Failure in the སློབ་གསོ་སྨྲ་བའི་སྡེ། article: 0
tbmgar_ALL_content_སློབ་གསོ་སྨྲ་བའི་སྡེ།.json
Successfully saved: tbmgar_ALL_content_སློབ་གསོ་སྨྲ་བའི་སྡེ།.json


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



 54%|█████▍    | 156/289 [09:55<07:37,  3.44s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



 54%|█████▍    | 157/289 [09:55<05:48,  2.64s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.

100%|██████████| 18/18 [02:19<00:00,  7.73s/it]


Total Failure in the སྔོན་བྱོན་སྐྱེས་ཆེན། article: 0
tbmgar_ALL_content_སྔོན་བྱོན་སྐྱེས་ཆེན།.json
Successfully saved: tbmgar_ALL_content_སྔོན་བྱོན་སྐྱེས་ཆེན།.json


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.







 56%|█████▌    | 289/519 [28:39<13:01,  3.40s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



 55%|█████▍    | 158/289 [10:01<07:40,  3.51s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.







 56%|█████▌    | 290/519 [28:40<11:00,  2.88s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.







 56%|█████▌    | 291/519 [28:41<08:45,  2.30s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


 41%|████▏     | 207/500 [22:08<25:28,  5.22s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.










 73%|███████▎  | 355/489 [32:47<12:58,  5.81s/it]Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.










 73%|███████▎  | 356/489 [32:48<09:50,  4.44s/it]Some characters could n

Total Failure in the རྒྱལ་ནང་གསར་འགྱུར། article: 0
tbmgar_ALL_content_རྒྱལ་ནང་གསར་འགྱུར།.json
Successfully saved: tbmgar_ALL_content_རྒྱལ་ནང་གསར་འགྱུར།.json
Processed 62 files
CPU times: user 32min 46s, sys: 1min 4s, total: 33min 51s
Wall time: 46min 31s


In [ ]:
# 14:44

In [ ]:
json_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.json')]
json_files